In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 加载预训练模型和分词器
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

# 将模型移动到 GPU（如果可用）
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((2048,), eps=1e-06)
    (rotary_emb):

In [4]:
import re

def generate_health_advice(instruction, input_text):
    # 将指令和输入拼接成完整输入
    full_input = f"{instruction} 输入：{input_text}"
    
    # 对输入进行编码
    inputs = tokenizer(
        full_input,
        return_tensors="pt",
        max_length=256,
        truncation=True,
        padding="max_length"
    )
    
    # 确保输入张量在正确的设备上
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # 使用 num_beams > 1 并移除 early_stopping
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=256,  # 增加最大生成长度
        num_beams=4,         # 使用束搜索
        do_sample=True,      # 启用采样
        temperature=0.7,     # 控制随机性
        top_p=0.9,           # 核采样
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # 解码生成的 token
    advice = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 提取生成的实际建议部分
    advice = advice.replace(full_input, "").strip()
    return advice

In [5]:
# 示例输入
example_instruction = (
    "您是经过培训的宠物个性化医疗保健代理，可以根据宠物的症状和体征，提供详细的诊断建议和护理指南。"
    "请分析以下输入中的症状，列出可能的原因，并给出具体的处理步骤和注意事项。"
)
example_input = (
    "最近的观察表明，动物类型:Dog 品种:Labrador 年龄:4岁 性别:Male 体重:25.0公斤, "
    "症状1:Fever 症状2:Lethargy 症状3:Appetite Loss 症状4:Vomiting 持续时间:3 days。"
    "食欲减退:Yes 呕吐:Yes 腹泻:No 咳嗽:No 呼吸困难:No 关节疼痛:No 皮肤病变:No 鼻分泌物:No 眼分泌物:No。"
    "我家的狗目前体温:39.5°C 心率:120，持续发热嗜睡"
)

# 分割输入文本为句子
sentences = [s.strip() for s in re.split(r'[。！？]', example_input) if s.strip()]

# 逐句生成健康建议
results = []
for sentence in sentences:
    advice = generate_health_advice(example_instruction, sentence)
    results.append((sentence, advice))

# 输出结果
for i, (sentence, advice) in enumerate(results):
    print(f"句子 {i+1}: {sentence}")
    print(f"健康建议: {advice}\n")

句子 1: 最近的观察表明，动物类型:Dog 品种:Labrador 年龄:4岁 性别:Male 体重:25.0公斤, 症状1:Fever 症状2:Lethargy 症状3:Appetite Loss 症状4:Vomiting 持续时间:3 days
健康建议: 一只4岁的成年雄性拉布拉多犬表现出发热、嗜睡、食欲不振和呕吐的症状，已经持续了3天。 根据您提供的信息，这只4岁成年男性的Labrador犬出现了发热（症状1）、嗜睡（症状2）、食欲减退（症状3）和呕吐（症状4）的症状。这些症状可能由多种原因引起，包括但不限于以下几种情况：

### 可能的原因：
1. **感染性疾病**：如细菌性或病毒性感染（例如犬细小病毒、犬瘟热等）、寄生虫感染（如钩虫、绦虫等）。
2. **消化系统问题**：胃肠道疾病（如胃炎、肠炎、食物中毒等）也可能导致上述症状。
3. **代谢或内分泌疾病**：例如糖尿病、甲状腺功能异常等。
4. **应激反应**：突然的环境变化或应激也可能导致类似的症状。

### 处理步骤：
- **立即就医**：鉴于症状已经持续3天，建议尽快联系兽医进行专业诊断和治疗。
- **记录症状**：详细记录宠物的体温变化、呕吐次数、食欲情况等，以便向兽医提供详细信息

句子 2: 食欲减退:Yes 呕吐:Yes 腹泻:No 咳嗽:No 呼吸困难:No 关节疼痛:No 皮肤病变:No 鼻分泌物:No 眼分泌物:No
健康建议: 宠物年龄:成年 体重:10公斤

根据您提供的症状，宠物可能患有胃肠道疾病。以下是一些可能的原因和处理步骤：

可能的原因：
1. 胃炎或胃溃疡：胃黏膜受到刺激或损伤，导致胃部不适和呕吐。
2. 肠炎或肠梗阻：肠道受到感染或阻塞，导致消化道功能紊乱，引起呕吐和食欲减低。
3. 食物过敏或不耐受：宠物对某些食物成分过敏或不能耐受，引发胃肠道反应。
4. 感染性疾病：细菌、病毒或寄生虫感染可引起胃肠道炎症，导致上述症状。
5. 其他疾病：如胰腺炎、肝脏疾病等也可能引起类似症状。

处理步骤及注意事项：

1. 观察并记录症状：注意观察宠物的呕吐频率、呕吐物的性质（如是否有血丝）、排泄物的颜色和质地等，以便向兽医提供详细信息。

2. 避免刺激性食物：在宠物恢复期间，避免给宠物喂食油腻、辛辣或难以消化的食物，以免加重胃肠道负担。

3. 补充水分